In [3]:
 pip install git+https://github.com/Wauplin/setfit@dont-use-deprecated-dataset-filter

  Cloning https://github.com/Wauplin/setfit (to revision dont-use-deprecated-dataset-filter) to /tmp/pip-req-build-cgjn7574
  Running command git clone --filter=blob:none --quiet https://github.com/Wauplin/setfit /tmp/pip-req-build-cgjn7574
  Running command git checkout -b dont-use-deprecated-dataset-filter --track origin/dont-use-deprecated-dataset-filter
  Switched to a new branch 'dont-use-deprecated-dataset-filter'
  Branch 'dont-use-deprecated-dataset-filter' set up to track remote branch 'dont-use-deprecated-dataset-filter' from 'origin'.
  Resolved https://github.com/Wauplin/setfit to commit 74bfc124b79c3c2820c465817c5aaa34dafdc5a7
  Preparing metadata (setup.py) ... done


In [4]:
!pip install setfit

In [5]:
from setfit import SetFitModel, Trainer, SetFitTrainer, TrainingArguments, sample_dataset , Trainer
from sklearn.model_selection import train_test_split
import pandas as pd
import re
from datasets import Dataset
from sklearn.metrics import f1_score
from sentence_transformers.losses import CosineSimilarityLoss
import numpy as np

In [26]:
df = pd.read_excel('/content/data66.xlsx')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
df.head(2)

,review_id,label,Topic,adjusted_comment
0,360948,Negative,التوصيل,تقيمه هنا في معروف تاخيرفي التوصيل
1,353856,Negative,التوصيل,تاخير في توصيل الطلب


In [28]:
df = df[['adjusted_comment' , 'Topic']]

In [29]:
df.rename(columns={'adjusted_comment': 'text'}, inplace=True)

In [30]:
df.rename(columns={'Topic': 'label'}, inplace=True)

In [31]:
df['label'].replace('الخدمه' , 'الخدمة' ,inplace=True)
df['label'].replace('الجوده' , 'الجودة' ,inplace=True)
df['label'].replace('المصداقية' , 'المصداقيه' ,inplace=True)

In [32]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 100
})

In [33]:
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Function to sample equal number of examples from each label
def sample_dataset(df, label_column="label", num_samples=10):
    # Sample equal number of examples from each class
    sampled_df = df.groupby(label_column).sample(n=num_samples, random_state=42, replace=True).reset_index(drop=True)
    return Dataset.from_pandas(sampled_df)

# Assuming df is your cleaned dataset in Pandas DataFrame format
# Step 1: Split the dataset into train_val (85%) and test (15%)
train_val_split, test_data = train_test_split(df, test_size=0.15, stratify=df['label'], random_state=42)

# Step 2: Further split train_val into train (85%) and validation (15%)
train_data, val_data = train_test_split(train_val_split, test_size=0.18, stratify=train_val_split['label'], random_state=42)

# Step 3: Sample 10 examples from each label in each dataset
train_dataset_sampled = sample_dataset(train_data, label_column="label", num_samples=8)
val_dataset_sampled = sample_dataset(val_data, label_column="label", num_samples=6)
test_dataset_sampled = sample_dataset(test_data, label_column="label", num_samples=6)

# Display the sizes of the sampled datasets
print(f"Training set: {len(train_dataset_sampled)}, Validation set: {len(val_dataset_sampled)}, Test set: {len(test_dataset_sampled)}")


Training set: 40, Validation set: 30, Test set: 30


In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(y_pred, y_test):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')  # 'macro', 'micro', or 'weighted'
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [35]:
from setfit import SetFitModel, SetFitTrainer
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load a pre-trained SetFit model and move it to GPU
model = SetFitModel.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


SetFitModel(model_body=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_head=LogisticRegression(), multi_target_strategy=None, normalize_embeddings=False, labels=None, model_card_data=SetFitModelCardData(language=None, license=None, tags=['setfit', 'sentence-transformers', 'text-classification', 'generated_from_setfit_trainer'], model_name='SetFit with nlptown/bert-base-multilingual-uncased-sentiment', model_id=None, dataset_name=None, dataset_id=None, dataset_revision=None, task_name=None, st_id='nlptown/bert-base-multilingual-uncased-sentiment', hyperparameters={}, eval_results_dict={}, eval_lines_list=[], metri

In [36]:
# Initialize the SetFitTrainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset_sampled,
    eval_dataset=val_dataset_sampled,
    metric=compute_metrics,
    batch_size=2,                      # Provided batch size
    num_epochs=2,                       # Provided number of epochs
    num_iterations=10,                  # Provided number of iterations
    learning_rate=1.0209298548906442e-05,  # Provided learning rate
    seed=2                              # Provided seed for reproducibility
)

<ipython-input-36-c8daab63ba7e>:2: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [23]:
!pip uninstall -y transformers

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2


In [24]:
!pip install transformers==4.39.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1


In [1]:
!pip uninstall -y huggingface_hub

Found existing installation: huggingface-hub 0.24.6
Uninstalling huggingface-hub-0.24.6:
  Successfully uninstalled huggingface-hub-0.24.6


In [2]:
!pip install huggingface_hub==0.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 8.9 MB/s eta 0:00:00


In [37]:
# Train the model
trainer.train()

# Evaluate the model on the test dataset
metrics = trainer.evaluate()

# Print all the metrics
print(f"Accuracy: {metrics['accuracy']}")
print(f"Precision: {metrics['precision']}")
print(f"Recall: {metrics['recall']}")
print(f"F1 Score: {metrics['f1']}")

***** Running training *****
  Num unique pairs = 800
  Batch size = 2
  Num epochs = 2
  Total optimization steps = 800


Step,Training Loss


***** Running evaluation *****


Accuracy: 0.9666666666666667
Precision: 0.9714285714285715
Recall: 0.9666666666666668
F1 Score: 0.9664335664335664


In [40]:
df_input =  pd.read_excel('/content/data66.xlsx')

In [42]:
df_input = df_input[['adjusted_comment' , 'Topic']]

In [43]:
df_input.rename(columns={'adjusted_comment': 'text'}, inplace=True)

In [44]:
df_input.rename(columns={'Topic': 'label'}, inplace=True)

In [45]:
df_input['label'].replace('الخدمه' , 'الخدمة' ,inplace=True)
df_input['label'].replace('الجوده' , 'الجودة' ,inplace=True)
df_input['label'].replace('المصداقية' , 'المصداقيه' ,inplace=True)

In [46]:
# model = SetFitModel.from_pretrained("best_model_2")

# Load the DataFrame from the CSV file
#df_input =  pd.read_excel('/content/data66.xlsx')

# Predict the sentiment for the input DataFrame
preds = model.predict(df_input['text'])

# Add the predicted sentiments to the DataFrame
df_input['Topic'] = preds

In [47]:
acc = accuracy_score(df_input['label'], df_input['Topic'])
print("acc:", acc)

precisionscore = precision_score(df_input['label'], df_input['Topic'] , average='macro')
print("pre:", precisionscore)

recall = recall_score(df_input['label'], df_input['Topic'] , average='macro')
print("recall:", recall)

f1 = f1_score(df_input['label'], df_input['Topic'] , average='macro')
print("f1:", f1)

acc: 0.9
pre: 0.9069264069264069
recall: 0.9
f1: 0.8966641201935319


In [ ]:
# model.save_pretrained("./models/Food_model")

In [48]:
df_total = pd.read_csv('/content/Cleaned_Comments__More_than_Two_Words_.csv' , index_col=0)
# df_total.drop(columns='Unnamed: 0' ,inplace=True)

In [49]:
df_total

,review_id,label,split_comment
0,360948,Negative,غلطتي ما جيت شفت تقيمه هنا في معروف تاخير
1,360948,Negative,تعامل سيء لابعد حدود الله لا يسامحهم
3,356339,Negative,اسواء خدمه عملاء محد يرد عليك تاخير ف الطلب
4,356339,Negative,محد يرد عليك
5,353856,Negative,اسوء خدمه عملاء تاخير في توصيل الطلب
...,...,...,...
21563,54465,Positive,موسه عموله من افضل الموسات بالتعامل مع مسوقاتها
21564,54465,Positive,مصداقيتها بتوصيل الطلبات لعملائها بوقت قصير
21565,54465,Positive,اهتمامها بتغليف المنتجات
21566,54465,Positive,الحرص عليها من التلف اثناء عمليه الشحن بمعنى ا...


In [ ]:
# df_total.drop(columns='comment' , inplace=True)
# df_total.drop(columns='label.1' , inplace=True)

In [50]:
df_total

,review_id,label,split_comment
0,360948,Negative,غلطتي ما جيت شفت تقيمه هنا في معروف تاخير
1,360948,Negative,تعامل سيء لابعد حدود الله لا يسامحهم
3,356339,Negative,اسواء خدمه عملاء محد يرد عليك تاخير ف الطلب
4,356339,Negative,محد يرد عليك
5,353856,Negative,اسوء خدمه عملاء تاخير في توصيل الطلب
...,...,...,...
21563,54465,Positive,موسه عموله من افضل الموسات بالتعامل مع مسوقاتها
21564,54465,Positive,مصداقيتها بتوصيل الطلبات لعملائها بوقت قصير
21565,54465,Positive,اهتمامها بتغليف المنتجات
21566,54465,Positive,الحرص عليها من التلف اثناء عمليه الشحن بمعنى ا...


In [52]:
# Remove rows where the text length in 'comment_split' is less than 5 characters
df_total = df_total[df_total['split_comment'].apply(lambda x: len(str(x)) >= 5)]

# Optionally, reset the index after removing the rows
df_total.reset_index(drop=True, inplace=True)

# Display the updated DataFrame
df_total.head()

# Remove rows that contain null values
df_total.dropna(inplace=True)

# Optionally, reset the index after removing the rows with null values
df_total.reset_index(drop=True, inplace=True)

# Display the updated DataFrame
df_total.head()


,review_id,label,split_comment
0,360948,Negative,غلطتي ما جيت شفت تقيمه هنا في معروف تاخير
1,360948,Negative,تعامل سيء لابعد حدود الله لا يسامحهم
2,356339,Negative,اسواء خدمه عملاء محد يرد عليك تاخير ف الطلب
3,356339,Negative,محد يرد عليك
4,353856,Negative,اسوء خدمه عملاء تاخير في توصيل الطلب


In [54]:
 probs = model.predict_proba(df_total['split_comment'])

 # Assuming you have 5 classes, create a column for each class probability
 df_total['التوصيل'] = probs[:, 0]
 df_total['الجوده'] = probs[:, 1]
 df_total['الخدمة'] = probs[:, 2]
 df_total['السعر'] = probs[:, 3]
 df_total['المصداقيه'] = probs[:, 4]

# # Optionally, you can also add the predicted class (most probable class) to the DataFrame
 df_total['Predicted_Class'] = probs.argmax(axis=1)

# # Print the DataFrame with the predicted probabilities and class
 df_total.head(50)

,review_id,label,split_comment,التوصيل,الجوده,الخدمة,السعر,المصداقيه,Predicted_Class
0,360948,Negative,غلطتي ما جيت شفت تقيمه هنا في معروف تاخير,0.177760,0.020041,0.029636,0.017200,0.755363,4
1,360948,Negative,تعامل سيء لابعد حدود الله لا يسامحهم,0.991847,0.002513,0.002273,0.001814,0.001552,0
2,356339,Negative,اسواء خدمه عملاء محد يرد عليك تاخير ف الطلب,0.021499,0.880301,0.013283,0.013449,0.071469,1
3,356339,Negative,محد يرد عليك,0.033943,0.937730,0.006861,0.007555,0.013911,1
4,353856,Negative,اسوء خدمه عملاء تاخير في توصيل الطلب,0.001996,0.993778,0.001322,0.001252,0.001652,1
5,353856,Negative,عدم الرد في اي موقع على العميل او افادته التعا...,0.004312,0.987520,0.002399,0.002511,0.003259,1
6,353856,Negative,ليس لديها اي,0.055827,0.166767,0.053463,0.659437,0.064506,3
7,353856,Negative,سيله تواصل عدم تجاوب مع العميل ابدا,0.858082,0.085812,0.011647,0.008963,0.035496,0
8,345234,Negative,اسواء متجر لايوجد اي مصداقيه طلبت من عشر ايام,0.002969,0.005147,0.002760,0.003143,0.985980,4
9,345234,Negative,ارسلولي رساله اعتذار,0.103300,0.091880,0.066431,0.080047,0.658342,4


In [55]:
import pandas as pd

# Define your class mapping
class_names = {
    'التوصيل': "التوصيل",
    'الجوده': "الجوده",
    'الخدمة': "الخدمة",
    'السعر': "السعر",
    'المصداقيه' : 'المصداقيه'
}

# Step 1: Find the maximum probability for each row
df_total['Max_Probability'] = df_total[['التوصيل'	 ,'الجوده' ,	'الخدمة'	,'السعر'	,'المصداقيه']].max(axis=1)

# Step 2: Get the predicted class (column name) with the highest probability
df_total['Predicted_Class'] = df_total[['التوصيل'	 ,'الجوده' ,	'الخدمة'	,'السعر'	,'المصداقيه']].idxmax(axis=1)

# Step 3: Apply the threshold: if Max_Probability is lower than 0.75, assign "غير ذلك"
threshold = 0.70
df_total['Label'] = df_total.apply(lambda row: class_names[row['Predicted_Class']] if row['Max_Probability'] >= threshold else "غير ذلك", axis=1)

# Display the DataFrame with the final predicted label
df_total


,review_id,label,split_comment,التوصيل,الجوده,الخدمة,السعر,المصداقيه,Predicted_Class,Max_Probability,Label
0,360948,Negative,غلطتي ما جيت شفت تقيمه هنا في معروف تاخير,0.177760,0.020041,0.029636,0.017200,0.755363,المصداقيه,0.755363,المصداقيه
1,360948,Negative,تعامل سيء لابعد حدود الله لا يسامحهم,0.991847,0.002513,0.002273,0.001814,0.001552,التوصيل,0.991847,التوصيل
2,356339,Negative,اسواء خدمه عملاء محد يرد عليك تاخير ف الطلب,0.021499,0.880301,0.013283,0.013449,0.071469,الجوده,0.880301,الجوده
3,356339,Negative,محد يرد عليك,0.033943,0.937730,0.006861,0.007555,0.013911,الجوده,0.937730,الجوده
4,353856,Negative,اسوء خدمه عملاء تاخير في توصيل الطلب,0.001996,0.993778,0.001322,0.001252,0.001652,الجوده,0.993778,الجوده
...,...,...,...,...,...,...,...,...,...,...,...
13251,54465,Positive,موسه عموله من افضل الموسات بالتعامل مع مسوقاتها,0.169319,0.107018,0.030711,0.022235,0.670717,المصداقيه,0.670717,غير ذلك
13252,54465,Positive,مصداقيتها بتوصيل الطلبات لعملائها بوقت قصير,0.077419,0.681781,0.014752,0.011271,0.214776,الجوده,0.681781,غير ذلك
13253,54465,Positive,اهتمامها بتغليف المنتجات,0.475620,0.353133,0.054418,0.050416,0.066413,التوصيل,0.475620,غير ذلك
13254,54465,Positive,الحرص عليها من التلف اثناء عمليه الشحن بمعنى ا...,0.747815,0.197536,0.022645,0.012139,0.019865,التوصيل,0.747815,التوصيل


In [56]:
df_total.drop(columns=['Predicted_Class' , 'Max_Probability', 'السعر' , 'المصداقيه' , 'الخدمة' , 'التوصيل' , 'الجوده'] , inplace=True)
#df_total.drop(columns=['التوصيل' , 'الجوده' , 'السعر' , 'المصداقيه'] , inplace=True)

In [ ]:
df_total

,BusinessId,review_id,rating,creation_date,text_length,label,BusinessType,comment_split,Label
0,60202.0,364189.0,1.0,2024-05-11 05:56:29.327,28,Negative,إلكترونيات,كنت اتعامل معهم اسعارهم ارخص,الخدمة
1,60202.0,364189.0,1.0,2024-05-11 05:56:29.327,245,Negative,إلكترونيات,اخر مره حصلت لي عمليه نصب على اساس الزباين غشيمين,التنسيق
2,60202.0,364189.0,1.0,2024-05-11 05:56:29.327,245,Negative,إلكترونيات,جهله اشتريت مكيف,غير ذلك
3,60202.0,364189.0,1.0,2024-05-11 05:56:29.327,245,Negative,إلكترونيات,سالت الموظف هل ه,المصداقيه
4,60202.0,364189.0,1.0,2024-05-11 05:56:29.327,245,Negative,إلكترونيات,فعلا قال ايوا يابيه متاكد,المصداقيه
...,...,...,...,...,...,...,...,...,...
22534,27286.0,53586.0,5.0,2018-01-23 02:03:24.150,20,Positive,إلكترونيات,موثوقه,المصداقيه
22535,27286.0,53364.0,5.0,2018-01-14 23:23:21.630,10,Positive,إلكترونيات,خدمه سريعه,الخدمة
22536,27286.0,53364.0,5.0,2018-01-14 23:23:21.630,7,Negative,إلكترونيات,ويستاهل,غير ذلك
22537,27286.0,53337.0,5.0,2018-01-13 01:38:23.183,24,Positive,إلكترونيات,خدمه سريعه,الخدمة


In [57]:
df_total.to_csv('3-Beauty_health_label_Topic.csv')

In [58]:
# Assuming you have already trained your model and saved it to "model"

# Load the trained model
# model = SetFitModel.from_pretrained("best_model_2")

# Input text
text = " انتظر طلبيتي من شهر ولم ازود باي معلومه عن الطلبيه لا رقم تبع لشحنه ولا وقت التسليم ولا حتى اسم شركه التوصيل اضافه الى سوء خدمه العملاء وعدم الاحترافيه في الهعمل ومع ذلك مستمرين في الاعلانات والعروض لاسف اذا انت مو قد ضغط الطلبات لا تروج عن منتجاتك وتخسر العملاء والا اهم شي عندكم الفلوس ؟"

# Predict the sentiment
sentiment = model.predict([text])

# Print the predicted sentiment
print(sentiment)

['التوصيل']
